In [ ]:
import DataReader as dr
import CollaborativeFiltering
import ContentBasedRecommender
import GraphBasedRecommender
import RunMe
import Evaluator
import logging as log
from Enumeration import Test_Mode
import pandas as pd
import numpy as np

In [ ]:
client_id = '048'

collaborative_result = []
# Loop the test
for seed in range(0, 5):
    # Use a different seed to choose different candidates and opporunity for each run

    # Comment or uncomment to test collaborative filtering
    collaborative_result.append(RunMe.runCollaborativeFiltering('data/app_'+client_id+'.csv',
                                                'data/opp_'+client_id+'.csv',
                                                Test_Mode.Warm_Candidate, seed))

In [ ]:
reciprocal_result = []
# Loop the test
for seed in range(0, 5):
    # Use a different seed to choose different candidates and opporunity for each run

    reciprocal_result.append(RunMe.runGraphBasedRecommender('data/app_'+client_id+'_doc2vec.csv',
                                           'data/opp_'+client_id+'_doc2vec.csv',
                                           'data/cv_'+client_id+'.csv',
                                           Test_Mode.Warm_Candidate,
                                           seed))

In [ ]:
def getTestData(app_filename, opp_filename, test_mode, seed):
    datareader = dr.CSVReader(app_filename, opp_filename)

    if test_mode == Test_Mode.Random_Candidate:
        train_app_df, gold_app_df, opp_df = datareader.getTrainNTestData_RandomCandidate(number_of_test_opp=500,
                                                                                         number_of_test_candidate=50,
                                                                                         seed=seed)
    elif test_mode == Test_Mode.Warm_Candidate:
        train_app_df, gold_app_df, opp_df = datareader.getTrainNTestData_WarmCandidate(number_of_test_opp=500,
                                                                                       number_of_test_candidate=50,
                                                                                       min_application=2,
                                                                                       application_train=1,
                                                                                       seed=seed)
    elif test_mode == Test_Mode.Cold_Candidate:
        train_app_df, gold_app_df, opp_df = datareader.getTrainNTestData_ColdCandidate(number_of_test_opp=500,
                                                                                       number_of_test_candidate=50,
                                                                                       seed=seed)
    elif test_mode == Test_Mode.Cold_Opportunity:
        train_app_df, gold_app_df, opp_df = datareader.getTrainNTestData_ColdOpportunity(number_of_test_opp=500,
                                                                                         number_of_test_candidate=50,
                                                                                         seed=seed)
    else:
        log.error('Invalid mode selected.')
        assert(False)

    return train_app_df, gold_app_df, opp_df

In [ ]:
ave_serendipity_list = []
for i in range(len(collaborative_result)):
    train_app_df, gold_app_df, opp_df = getTestData('data/app_'+client_id+'_doc2vec.csv', 
                                                      'data/opp_'+client_id+'_doc2vec.csv', 
                                                      Test_Mode.Warm_Candidate, 
                                                      i)
    col_rec_dict = collaborative_result[i][2]
    reci_rec_dict = reciprocal_result[i][2]
    
    unique_candidate_list = list(col_rec_dict.keys())
    
    total_unexpected = 0
    total_useful = 0
    total_serendipity = 0
    serendipity_list = []
    for candidate in unique_candidate_list:
        col_rec_set = set(col_rec_dict[candidate])
        reci_rec_set = set(reci_rec_dict[candidate])

        unexpected = reci_rec_set - col_rec_set
        gold_application_list = gold_app_df[gold_app_df.candidate_id == candidate].opp_id.values
        
        unexpected_count = len(unexpected)
        total_unexpected += unexpected_count
        useful_count = len(set(gold_application_list) - unexpected)
        total_useful += useful_count
        
        if unexpected_count != 0:
            serendipity_list.append(useful_count/unexpected_count)
#         serendipity = useful_count/unexpected_count
#         total_serendipity += serendipity
        
    ave_serendipity = np.mean(serendipity_list)
    print('ave: %f' %ave_serendipity)
    ave_serendipity_list.append(ave_serendipity)
    

print('Average Serendipity for Client %s: %.3f' %(client_id, np.mean(ave_serendipity_list)))